In [44]:
# stdlib
from dataclasses import dataclass
import uuid
from uuid import UUID as uuid_type

# third party
from nacl.encoding import HexEncoder
from nacl.signing import SigningKey

In [69]:
@dataclass
class Domain:
    node_uid: uuid_type

In [74]:
d1 = Domain(uuid.uuid4())
d1

Domain(node_uid=UUID('81d4b35d-68ca-47d1-8b24-4e60d77cfeec'))

In [75]:
d2 = Domain(uuid.uuid4())
d2

Domain(node_uid=UUID('fd34b9fc-9424-4aa7-ade3-270fa7f25b56'))

Domain(node_uid=UUID('fe50eb36-37c9-439b-aec7-276e56092662'))

In [26]:
class NUID:
    def __init__(self, node_uid, obj_uid) -> None:
        self.node_uid = node_uid
        self.obj_uid = obj_uid

    @classmethod
    def new(cls) -> NUID:
        return cls(node_uid=uuid.uuid4(), obj_uid=uuid.uuid4())

    def __repr__(self) -> str:
        return f"NUID: {self.obj_uid}@<{self.node_uid}>"

In [60]:
key2 = SigningKey.generate()
key2

In [39]:
key = SigningKey.generate()
key

In [41]:
verify_key = key.verify_key
verify_key

In [62]:
verify_key2 = key2.verify_key
verify_key2

In [47]:
verify_key_str = verify_key.encode(encoder=HexEncoder).decode("utf-8")
verify_key_str

'ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'

In [63]:
verify_key_str2 = verify_key2.encode(encoder=HexEncoder).decode("utf-8")
verify_key_str2

'8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662'

In [304]:
@dataclass
class User:
    verify_key: str

    def __hash__(self) -> str:
        return hash(self.verify_key)

In [305]:
u1 = User(verify_key=verify_key_str)
u1

User(verify_key='ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8')

In [306]:
u2 = User(verify_key=verify_key_str2)
u2

User(verify_key='8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662')

In [307]:
data_nuid = NUID(d1.node_uid, uuid.uuid4())
data_nuid

NUID: a11a5048-099a-4597-bda3-a1bd0f4d53ed@<81d4b35d-68ca-47d1-8b24-4e60d77cfeec>

In [308]:
@dataclass
class SyftObject:
    id: NUID
    value: int

In [309]:
t = SyftObject(id=data_nuid, value=1)

In [310]:
model_nuid = NUID(d2.node_uid, uuid.uuid4())
model_nuid

NUID: b46acc56-4178-41ca-8426-874f968f1254@<fd34b9fc-9424-4aa7-ade3-270fa7f25b56>

In [311]:
m = SyftObject(id=model_nuid, value=2)

In [312]:
store[m.id.obj_uid] = m

In [313]:
store[t.id.obj_uid] = t

In [314]:
store

{UUID('6971d4ec-6b2b-4f36-8c37-ac18eeefcb45'): SyftObject(id=NUID: 6971d4ec-6b2b-4f36-8c37-ac18eeefcb45@<81d4b35d-68ca-47d1-8b24-4e60d77cfeec>, value=1),
 UUID('af69c65b-55b5-4eab-aa4b-c47db9137b46'): SyftObject(id=NUID: af69c65b-55b5-4eab-aa4b-c47db9137b46@<fd34b9fc-9424-4aa7-ade3-270fa7f25b56>, value=2),
 UUID('709cc26a-d7ea-4da6-ba86-f1895b5b4399'): Code(id=UUID('709cc26a-d7ea-4da6-ba86-f1895b5b4399'), func=<function add at 0x108473b00>),
 UUID('b46acc56-4178-41ca-8426-874f968f1254'): SyftObject(id=NUID: b46acc56-4178-41ca-8426-874f968f1254@<fd34b9fc-9424-4aa7-ade3-270fa7f25b56>, value=2),
 UUID('a11a5048-099a-4597-bda3-a1bd0f4d53ed'): SyftObject(id=NUID: a11a5048-099a-4597-bda3-a1bd0f4d53ed@<81d4b35d-68ca-47d1-8b24-4e60d77cfeec>, value=1)}

In [315]:
permissions[t.id.obj_uid] = set()

TypeError: 'NoneType' object does not support item assignment

In [316]:
permissions[m.id.obj_uid] = set()

TypeError: 'NoneType' object does not support item assignment

In [317]:
permissions

In [318]:
# stdlib
from enum import Enum

In [319]:
class ActionPermission(Enum):
    OWNER = 1
    READ = 2
    ALL_READ = 4
    WRITE = 8
    ALL_WRITE = 32
    EXECUTE = 64
    ALL_EXECUTE = 128

In [320]:
permissions[t.id.obj_uid].add(f"{ActionPermission.OWNER}_{u1.verify_key}")

TypeError: 'NoneType' object is not subscriptable

In [321]:
permissions

In [322]:
permissions[t.id.obj_uid].add(f"{ActionPermission.OWNER}_{u1.verify_key}")

TypeError: 'NoneType' object is not subscriptable

In [323]:
# stdlib
from typing import Callable

In [324]:
@dataclass
class Code:
    id: uuid_type
    func: Callable

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)

In [325]:
c(t, m)

3

In [326]:
store[c.id] = c

In [327]:
permissions[c.id] = set()

TypeError: 'NoneType' object does not support item assignment

In [328]:
permissions[c.id].add(f"{ActionPermission.OWNER}_{u1.verify_key}")
permissions[c.id].add(f"{ActionPermission.OWNER}_{u2.verify_key}")

TypeError: 'NoneType' object is not subscriptable

In [329]:
permissions[m.id.obj_uid].add(f"{ActionPermission.OWNER}_{u2.verify_key}")

TypeError: 'NoneType' object is not subscriptable

In [330]:
permissions

In [331]:
permissions

In [332]:
"""
- user 1 @ domain 1
    - result that only they can see
        - needs READ permission
    - input to code requires object from user 1 / domain 1
        - need WRITE permission for the UID of the store location
    - enclave executes code against data
        - requires all[n] parties EXECUTE permissions

"""

'\n- user 1 @ domain 1\n    - result that only they can see\n        - needs READ permission\n    - input to code requires object from user 1 / domain 1\n        - need WRITE permission for the UID of the store location\n    - enclave executes code against data\n        - requires all[n] parties EXECUTE permissions\n\n'

In [333]:
permissions

In [334]:
a = list(permissions.keys())[0]
"ActionPermission.OWNER_ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8" in permissions[
    a
]

AttributeError: 'NoneType' object has no attribute 'keys'

In [335]:
str(ActionPermission.OWNER)

'ActionPermission.OWNER'

In [336]:
permissions = None

In [446]:
@dataclass
class PermissionChange:
    uid: uuid_type
    store_uid: uuid_type
    permissions: list[str]
    requester: User
    shared_owners_approval: set[str]
    shared_owners: set[User]

    def approve(user, enclave):
        return enclave.approve_as_user(uid, user)

In [370]:
@dataclass
class Message:
    message: str
    linked_change_id: uuid_type | None
    from_user: User
    to_user: User

In [379]:
user_lookup = {
    u1.verify_key: u1,
    u2.verify_key: u2,
}

In [425]:
# domain keys versus user keys
@dataclass
class Enclave:
    node_uid: uuid_type
    owner_keys: list[uuid_type]
    permissions: dict
    store: dict
    change_queue: dict
    inbox: dict

    def obj_or_id(self, obj):
        if isinstance(obj, SyftObject):
            uid = obj.id.obj_uid
        else:
            uid = obj
        return uid

    def get_owners(self, obj) -> list[User]:
        uid = self.obj_or_id(obj)
        if not uid in self.permissions.keys():
            return []

        owners = []
        if uid in self.permissions.keys():
            for permission in self.permissions[uid]:
                permission_str = str(ActionPermission.OWNER)
                if permission_str in permission:
                    user_key = permission.split("_")[-1]
                    owners.append(user_lookup[user_key])
        return owners
    
    def has_multiple_owners(self, obj) -> bool:
        uid = self.obj_or_id(obj)
        if not uid in self.permissions.keys():
            return False
        
        if uid in self.permissions.keys():
            count = 0
            for permission in self.permissions[uid]:
                permission_str = str(ActionPermission.OWNER)
                if permission_str in permission:
                    count += 1
        return count > 1
    
    def check_owner(self, uid, user) -> bool:
        if not uid in self.permissions.keys():
            return True
        if uid in self.permissions.keys():
            return f"{ActionPermission.OWNER}_{user.verify_key}" in self.permissions[uid]

    def add_obj(self, obj, as_user):
        uid = obj.id.obj_uid
        if self.check_owner(uid, as_user):
            self.store[uid] = obj
            if uid not in self.permissions:
                self.permissions[uid] = set()
            self.permissions[uid].add(f"{ActionPermission.OWNER}_{as_user.verify_key}")
        else:
            print("cant add obj already exists or has a owner")

    def add_permission(self, obj, as_user, for_user, permission):
        uid = obj.id.obj_uid
        if self.check_owner(uid, as_user):
            if uid not in self.permissions:
                print("here?")
                self.permissions[uid] = set()

            if not self.has_multiple_owners(uid):
                print("permission added")
                self.permissions[uid].add(f"{ActionPermission.OWNER}_{for_user.verify_key}")
            else:
                print("multiple owners, joint permission requested")
                pc_uid = uuid.uuid4()
                all_users = []
                owners = self.get_owners(obj)
                for owner in owners:
                    all_users.append(owner)

                all_keys = set()
                all_keys.add(as_user.verify_key)

                pc = PermissionChange(uid=pc_uid, store_uid=uid, permissions=[f"{ActionPermission.OWNER}_{for_user.verify_key}"], requester=as_user, shared_owners=all_keys)
                self.change_queue[pc_uid] = pc
                
                for owner in owners:
                    if owner is not as_user:
                        self.send_message(message="Allow change?", from_user=as_user, to_user=owner, linked_change_id=pc_uid)
        else:
            print("cant change obj permissions you arent the owner")

    def has_permission(self, uid, user, permission) -> bool:
        if uid not in self.permissions:
            return False
        return f"{permission}_{user.verify_key}" in self.permissions[uid]

    def read(self, obj, as_user):
        uid = self.obj_or_id(obj)
        if self.has_permission(uid, as_user, ActionPermission.READ):
            return self.store[uid]
        return "Error: you dont have permission"

    def send_message(self, message, from_user, to_user, linked_change_id=None):
        msg = Message(message, linked_change_id, from_user, to_user)
        if to_user not in self.inbox:
            self.inbox[to_user] = []
        self.inbox[to_user].append(msg)

    def user_inbox(self, to_user):
        if to_user not in self.inbox:
            self.inbox[to_user] = []

        return self.inbox[to_user]

    def approve_as_user(self, uid, user):
        uid = self.obj_or_id(uid)
        change = change_queue[uid]
        change.shared_owners.add()
        if change

In [426]:
# domain 1 creates enclave
e1 = Enclave(
    uuid.uuid4(),
    owner_keys=[u1.verify_key, u2.verify_key],
    permissions={},
    store={},
    change_queue={},
    inbox={},
)
e1

Enclave(node_uid=UUID('b5774e95-02ce-4e68-a400-ecaeb57900a1'), owner_keys=['ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8', '8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662'], permissions={}, store={}, change_queue={}, inbox={})

In [427]:
e1.send_message("hi", u1, u2)

In [428]:
e1.user_inbox(u2)[0]

Message(message='hi', linked_change_id=None, from_user=User(verify_key='ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'), to_user=User(verify_key='8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662'))

In [429]:
e1.add_obj(t, u1)

In [430]:
u1, u2

(User(verify_key='ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'),
 User(verify_key='8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662'))

In [431]:
e1.has_multiple_owners(t)

False

In [432]:
e1.add_permission(t, u1, u2, permission=ActionPermission.OWNER)

permission added


In [433]:
e1.permissions

{UUID('a11a5048-099a-4597-bda3-a1bd0f4d53ed'): {'ActionPermission.OWNER_8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662',
  'ActionPermission.OWNER_ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'}}

In [434]:
e1.read(t, u1)

'Error: you dont have permission'

In [435]:
e1.permissions

{UUID('a11a5048-099a-4597-bda3-a1bd0f4d53ed'): {'ActionPermission.OWNER_8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662',
  'ActionPermission.OWNER_ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'}}

In [436]:
e1.has_multiple_owners(t)

True

In [437]:
e1.add_permission(t, u1, u1, permission=ActionPermission.READ)

multiple owners, joint permission requested


In [442]:
request = e1.user_inbox(u2)[0]

In [443]:
request

Message(message='hi', linked_change_id=None, from_user=User(verify_key='ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'), to_user=User(verify_key='8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662'))

In [444]:
request.approve(e1)

AttributeError: 'Message' object has no attribute 'approve'

In [441]:
e1.change_queue

{UUID('6caf614d-5830-488a-82f7-a9d12ff5b9b2'): PermissionChange(uid=UUID('6caf614d-5830-488a-82f7-a9d12ff5b9b2'), store_uid=UUID('a11a5048-099a-4597-bda3-a1bd0f4d53ed'), permissions=['ActionPermission.OWNER_ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'], requester=User(verify_key='ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'), shared_owners={'ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8'})}

In [363]:
a = list(e1.permissions.keys())[0]

In [246]:
for a in e1.permissions[a]:
    print(a)

ActionPermission.OWNER_ef968740eb671fc17939b5560326f280339c208b962c7965ba4ec3137906f8e8
ActionPermission.OWNER_8c675358c1b4ac08458cb94544a285351f5a3e255244d9a36338f001603f0662


In [124]:
# domain 1 sends in code


def add(a, b) -> int:
    return a.value + b.value


c = Code(id=uuid.uuid4(), func=add)

In [ ]:
# domain 1 populates initial permissions
# domain 2 confirms state agrees with their expectations

In [ ]:
# domain 2 data scientist tries to send in data

In [ ]:
# system eval loop
# get submitted code job
# check if able to execute
# run